# Text Classification Model for Movie Review Sentiment Analysis Using TensorFlow Take 5
### David Lowe
### December 25, 2020

Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: This project aims to construct a text classification model using a neural network and document the end-to-end steps using a template. The Movie Review Sentiment Analysis dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

Additional Notes: This script is a replication, with some small modifications, of Dr. Jason Brownlee's blog post, Deep Convolutional Neural Network for Sentiment Analysis (https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/). I plan to leverage Dr. Brownlee's tutorial and build a TensorFlow-based text classification notebook template for future modeling of similar datasets.

INTRODUCTION: The Movie Review Data is a collection of movie reviews retrieved from the imdb.com website in the early 2000s by Bo Pang and Lillian Lee. The reviews were collected and made available as part of their research on natural language processing. The dataset comprises 1,000 positive and 1,000 negative movie reviews drawn from an archive of the rec.arts.movies.reviews newsgroup hosted at IMDB. The authors refer to this dataset as the 'polarity dataset.'

We will use the last 100 positive reviews and the last 100 negative reviews as a test set (200 reviews) and the remaining 1,800 reviews as the training dataset. This is a 90% train, 10% split of the data.

In iteration Take1, we constructed the necessary code modules to handle the tasks of loading text, cleaning text, and vocabulary development.

In iteration Take2, we constructed a bag-of-words model and analyzed it with a simple multi-layer perceptron network.

In iteration Take3, we fine-tuned the model using different Tokenizer modes in the Keras API.

In iteration Take4, we will fine-tune the model using word embedding to fit a deep learning model.

In this Take5 iteration, we will fine-tune the model using the Word2Vec algorithm for developing a standalone word embedding mechanism.

ANALYSIS: From iteration Take2, the baseline model's performance achieved an average accuracy score of 85.81% after 25 epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 91.50%.

From iteration Take3, the best model's performance (using the frequency Tokenizer mode) achieved an average accuracy score of 86.06% after 25 epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 86.50%.

From iteration Take4, the best model's performance (using the word embedding with Keras) achieved an average accuracy score of 84.17% after ten epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 88.50%.

In this Take5 iteration, the best model's performance (using the Word2Vec word embedding algorithm) achieved an average accuracy score of 52.61% after ten epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 49.00%.

CONCLUSION: In this iteration, the Word2Vec word embedding algorithm appeared unsuitable for modeling this dataset. We should consider experimenting with other word embedding techniques for further modeling.

Dataset Used: Movie Review Sentiment Analysis Dataset

Dataset ML Model: Binary class text classification with text-oriented features

Dataset Reference: https://www.cs.cornell.edu/home/llee/papers/cutsent.pdf and http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

One potential source of performance benchmarks: https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

A deep-learning text classification project generally can be broken down into five major tasks:

1. Prepare Environment
2. Load and Prepare Text Data
3. Define and Train Models
4. Evaluate and Optimize Models
5. Finalize Model and Make Predictions

# Task 1 - Prepare Environment

In [1]:
# # Install the packages to support accessing environment variable and SQL databases
# !pip install python-dotenv PyMySQL boto3

In [2]:
# # Retrieve GPU configuration information from Colab
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#     print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
#     print('and then re-execute this cell.')
# else:
#     print(gpu_info)

In [3]:
# # Retrieve memory configuration information from Colab
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#     print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
#     print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
#     print('re-execute this cell.')
# else:
#     print('You are using a high-RAM runtime!')

In [4]:
# # Direct Colab to use TensorFlow v2
# %tensorflow_version 2.x

In [5]:
# Retrieve CPU information from the system
ncpu = !nproc
print("The number of available CPUs is:", ncpu[0])

The number of available CPUs is: 4


## 1.a) Load libraries and modules

In [6]:
# Set the random seed number for reproducible results
seedNum = 888

In [7]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
import boto3
import shutil
import string
import nltk
import gensim
from nltk.corpus import stopwords
from collections import Counter
from datetime import datetime
from dotenv import load_dotenv
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
tf.random.set_seed(seedNum)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [8]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package movie_reviews is a

True

## 1.b) Set up the controlling parameters and functions

In [9]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the number of CPU cores available for multi-thread processing
n_jobs = 2

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set Pandas options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# Set the percentage sizes for splitting the dataset
test_set_size = 0.2
val_set_size = 0.25

# Set the number of folds for cross validation
n_folds = 5
n_iterations = 1

# Set various default modeling parameters
default_loss = 'binary_crossentropy'
default_metrics = ['accuracy']
default_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
default_kernel_init = tf.keras.initializers.RandomNormal(seed=seedNum)
default_epochs = 10
default_batch_size = 16
default_vector_space = 100
default_filters = 128
default_kernel_size = 5
default_pool_size = 2
default_neighbors = 5

# Define the labels to use for graphing the data
train_metric = "accuracy"
validation_metric = "val_accuracy"
train_loss = "loss"
validation_loss = "val_loss"

# Check the number of GPUs accessible through TensorFlow
print('Num GPUs Available:', len(tf.config.list_physical_devices('GPU')))

# Print out the TensorFlow version for confirmation
print('TensorFlow version:', tf.__version__)

Num GPUs Available: 0
TensorFlow version: 2.3.1


In [10]:
# Set up the parent directory location for loading the dotenv files
# useColab = True
# if useColab:
#     # Mount Google Drive locally for storing files
#     from google.colab import drive
#     drive.mount('/content/gdrive')
#     gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
#     env_path = '/content/gdrive/My Drive/Colab Notebooks/'
#     dotenv_path = env_path + "python_script.env"
#     load_dotenv(dotenv_path=dotenv_path)

# Set up the dotenv file for retrieving environment variables
# useLocalPC = True
# if useLocalPC:
#     env_path = "/Users/david/PycharmProjects/"
#     dotenv_path = env_path + "python_script.env"
#     load_dotenv(dotenv_path=dotenv_path)

In [11]:
# Set up the email notification function
def status_notify(msg_text):
    access_key = os.environ.get('SNS_ACCESS_KEY')
    secret_key = os.environ.get('SNS_SECRET_KEY')
    aws_region = os.environ.get('SNS_AWS_REGION')
    topic_arn = os.environ.get('SNS_TOPIC_ARN')
    if (access_key is None) or (secret_key is None) or (aws_region is None):
        sys.exit("Incomplete notification setup info. Script Processing Aborted!!!")
    sns = boto3.client('sns', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=aws_region)
    response = sns.publish(TopicArn=topic_arn, Message=msg_text)
    if response['ResponseMetadata']['HTTPStatusCode'] != 200 :
        print('Status notification not OK with HTTP status code:', response['ResponseMetadata']['HTTPStatusCode'])

In [12]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [13]:
# Reset the random number generators
def reset_random(x):
    random.seed(x)
    np.random.seed(x)
    tf.random.set_seed(x)

In [14]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 2 - Load and Prepare Text Data

In [15]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 2.a) Download Text Data Archive

In [16]:
# Clean up the old files and download directories before receiving new ones
!rm -rf staging/
!rm review_polarity.tar.gz
!rm vocabulary.txt

In [17]:
!wget https://dainesanalytics.com/datasets/cornell-movie-review-polarity/review_polarity.tar.gz

--2020-12-13 00:26:39--  https://dainesanalytics.com/datasets/cornell-movie-review-polarity/review_polarity.tar.gz
Resolving dainesanalytics.com (dainesanalytics.com)... 13.225.150.103, 13.225.150.58, 13.225.150.31, ...
Connecting to dainesanalytics.com (dainesanalytics.com)|13.225.150.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  11.8MB/s    in 0.3s    

2020-12-13 00:26:39 (11.8 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



## 2.b) Splitting Data for Training and Validation

In [18]:
staging_dir = 'staging/'
!mkdir staging/
!mkdir staging/testing/
!mkdir staging/testing/pos/
!mkdir staging/testing/neg/

In [19]:
local_archive = 'review_polarity.tar.gz'
shutil.unpack_archive(local_archive, staging_dir)

In [20]:
training_dir = 'staging/txt_sentoken/'
testing_dir = 'staging/testing/'
classA_name = 'pos'
classB_name = 'neg'

In [21]:
# Brief listing of training text files for both classes before splitting
training_classA_dir = os.path.join(training_dir, classA_name)
training_classA_files = os.listdir(training_classA_dir)
print('Number of training images for', classA_name, ':', len(training_classA_files))
print('Training samples for', classA_name, ':', training_classA_files[:10])

training_classB_dir = os.path.join(training_dir, classB_name)
training_classB_files = os.listdir(training_classB_dir)
print('Number of training images for', classB_name, ':', len(training_classB_files))
print('Training samples for', classB_name, ':', training_classB_files[:10])

Number of training images for pos : 1000
Training samples for pos : ['cv000_29590.txt', 'cv001_18431.txt', 'cv002_15918.txt', 'cv003_11664.txt', 'cv004_11636.txt', 'cv005_29443.txt', 'cv006_15448.txt', 'cv007_4968.txt', 'cv008_29435.txt', 'cv009_29592.txt']
Number of training images for neg : 1000
Training samples for neg : ['cv000_29416.txt', 'cv001_19502.txt', 'cv002_17424.txt', 'cv003_12683.txt', 'cv004_12641.txt', 'cv005_29357.txt', 'cv006_17022.txt', 'cv007_4992.txt', 'cv008_29326.txt', 'cv009_29417.txt']


In [22]:
# Move the testing files from the training directories
testing_classA_dir = os.path.join(testing_dir, classA_name)
testing_classB_dir = os.path.join(testing_dir, classB_name)
file_prefix = 'cv9'

i = 0
for text_file in training_classA_files:
    if text_file.startswith(file_prefix):
        source_file = training_classA_dir + '/' + text_file
        dest_file = testing_classA_dir + '/' + text_file
#         print('Moving file from', source_file, 'to', dest_file)
        shutil.move(source_file, dest_file)
        i = i + 1
print('Number of', classA_name, 'files moved:', i, '\n')

i = 0
for text_file in training_classB_files:
    if text_file.startswith(file_prefix):
        source_file = training_classB_dir + '/' + text_file
        dest_file = testing_classB_dir + '/' + text_file
#         print('Moving file from', source_file, 'to', dest_file)
        shutil.move(source_file, dest_file)
        i = i + 1
print('Number of', classB_name, 'files moved:', i)

Number of pos files moved: 100 

Number of neg files moved: 100


In [23]:
# Brief listing of training text files for both classes after splitting
training_classA_files = os.listdir(training_classA_dir)
print('Number of training files for', classA_name, ':', len(training_classA_files))
print('Training samples for', classA_name, ':', training_classA_files[:10])

training_classB_files = os.listdir(training_classB_dir)
print('Number of training files for', classB_name, ':', len(training_classB_files))
print('Training samples for', classB_name, ':', training_classB_files[:10])

Number of training files for pos : 900
Training samples for pos : ['cv000_29590.txt', 'cv001_18431.txt', 'cv002_15918.txt', 'cv003_11664.txt', 'cv004_11636.txt', 'cv005_29443.txt', 'cv006_15448.txt', 'cv007_4968.txt', 'cv008_29435.txt', 'cv009_29592.txt']
Number of training files for neg : 900
Training samples for neg : ['cv000_29416.txt', 'cv001_19502.txt', 'cv002_17424.txt', 'cv003_12683.txt', 'cv004_12641.txt', 'cv005_29357.txt', 'cv006_17022.txt', 'cv007_4992.txt', 'cv008_29326.txt', 'cv009_29417.txt']


In [24]:
# Brief listing of testing text files for both classes after splitting
testing_classA_files = os.listdir(testing_classA_dir)
print('Number of testing files for', classA_name, ':', len(testing_classA_files))
print('Test samples for', classA_name, ':', testing_classA_files[:10])

testing_classB_files = os.listdir(testing_classB_dir)
print('Number of testing files for', classB_name, ':', len(testing_classB_files))
print('Test samples for', classB_name, ':', testing_classB_files[:10])

Number of testing files for pos : 100
Test samples for pos : ['cv900_10331.txt', 'cv901_11017.txt', 'cv902_12256.txt', 'cv903_17822.txt', 'cv904_24353.txt', 'cv905_29114.txt', 'cv906_11491.txt', 'cv907_3541.txt', 'cv908_16009.txt', 'cv909_9960.txt']
Number of testing files for neg : 100
Test samples for neg : ['cv900_10800.txt', 'cv901_11934.txt', 'cv902_13217.txt', 'cv903_18981.txt', 'cv904_25663.txt', 'cv905_28965.txt', 'cv906_12332.txt', 'cv907_3193.txt', 'cv908_17779.txt', 'cv909_9973.txt']


## 2.c) Load Document and Build Vocabulary

In [25]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [26]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [27]:
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

In [28]:
# load all docs in a directory
def build_vocab_from_docs(directory, vocab):
    # walk through all files in the folder
    i = 0
    print('Processing the text files and showing the first 10...')
    for filename in os.listdir(directory):
        # skip files that do not have the right extension
        if not filename.endswith(".txt"):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab)
        i = i + 1
        if i < 10: print('Loaded %s' % path)
    print('Total number of text files loaded:', i, '\n')

In [29]:
# define vocab
vocab = Counter()
# add all docs to vocab
build_vocab_from_docs(training_classA_dir, vocab)
build_vocab_from_docs(training_classB_dir, vocab)
# print the size of the vocab
print('The total number of words in the vocabulary:', len(vocab))
# print the top words in the vocab
top_words = 50
print('The top', top_words, 'words in the vocabulary:\n', vocab.most_common(top_words))

Processing the text files and showing the first 10...
Loaded staging/txt_sentoken/pos/cv000_29590.txt
Loaded staging/txt_sentoken/pos/cv001_18431.txt
Loaded staging/txt_sentoken/pos/cv002_15918.txt
Loaded staging/txt_sentoken/pos/cv003_11664.txt
Loaded staging/txt_sentoken/pos/cv004_11636.txt
Loaded staging/txt_sentoken/pos/cv005_29443.txt
Loaded staging/txt_sentoken/pos/cv006_15448.txt
Loaded staging/txt_sentoken/pos/cv007_4968.txt
Loaded staging/txt_sentoken/pos/cv008_29435.txt
Total number of text files loaded: 900 

Processing the text files and showing the first 10...
Loaded staging/txt_sentoken/neg/cv000_29416.txt
Loaded staging/txt_sentoken/neg/cv001_19502.txt
Loaded staging/txt_sentoken/neg/cv002_17424.txt
Loaded staging/txt_sentoken/neg/cv003_12683.txt
Loaded staging/txt_sentoken/neg/cv004_12641.txt
Loaded staging/txt_sentoken/neg/cv005_29357.txt
Loaded staging/txt_sentoken/neg/cv006_17022.txt
Loaded staging/txt_sentoken/neg/cv007_4992.txt
Loaded staging/txt_sentoken/neg/cv008

In [30]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print('The number of words with the minimum appearance:', len(tokens))

The number of words with the minimum appearance: 25767


In [31]:
# save list to file
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'vocabulary.txt')

## 2.d) Prepare Word Embedding using Word2Vec Algorithm

In [32]:
# load the vocabulary
vocab_filename = 'vocabulary.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print('Number of tokens in the vocabulary:', len(vocab))

Number of tokens in the vocabulary: 25767


In [33]:
# turn a doc into clean tokens
def doc_to_clean_lines(doc, vocab):
    clean_lines = list()
    lines = doc.splitlines()
    for line in lines:
        # split into tokens by white space
        tokens = line.split()
        # remove punctuation from each token
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        # filter out tokens not in vocab
        tokens = [w for w in tokens if w in vocab]
        clean_lines.append(tokens)
    return clean_lines

In [34]:
# load all docs in a directory
def load_lines_from_docs(directory, vocab):
    lines = list()
    # walk through all files in the folder
    for filename in os.listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        doc = load_doc(path)
        doc_lines = doc_to_clean_lines(doc, vocab)
        # add lines to list
        lines += doc_lines
    return lines

In [35]:
# load training data
positive_train_cases = load_lines_from_docs(training_classA_dir, vocab)
print('The number of positive sentences processed:', len(positive_train_cases))
negative_train_cases = load_lines_from_docs(training_classB_dir, vocab)
print('The number of negative sentences processed:', len(negative_train_cases))
train_sentences = negative_train_cases + positive_train_cases
print('Total training sentences: %d' % len(train_sentences))

The number of positive sentences processed: 29525
The number of negative sentences processed: 28584
Total training sentences: 58109


In [36]:
# train word2vec model
embed_model = gensim.models.Word2Vec(train_sentences, size=default_vector_space, window=default_neighbors, workers=n_jobs, min_count=1)
# summarize vocabulary size in model
words = list(embed_model.wv.vocab)
print('Vocabulary size: %d' % len(words))

Vocabulary size: 25767


In [37]:
# save model in ASCII (word2vec) format
filename = 'embedding_word2vec.txt'
embed_model.wv.save_word2vec_format(filename, binary=False)

## 2.e) Create Tokenizer and Encode the Input Text

In [38]:
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [39]:
# load all docs in a directory
def process_docs_to_lines(directory, vocab):
    lines = list()
    # walk through all files in the folder
    for filename in os.listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        line = doc_to_line(path, vocab)
        # add to list
        lines.append(line)
    return lines

In [40]:
# prepare encoding of docs
def encode_train_data(train_docs, maxlen):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    encoded_docs = tokenizer.texts_to_sequences(train_docs)
    # pad sequences
    train_encoded = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
    return train_encoded, tokenizer

In [41]:
# Load all training cases
positive_train_cases = process_docs_to_lines(training_classA_dir, vocab)
print('The number of positive reviews processed:', len(positive_train_cases))
negative_train_cases = process_docs_to_lines(training_classB_dir, vocab)
print('The number of negative reviews processed:', len(negative_train_cases))
training_docs =  negative_train_cases + positive_train_cases

The number of positive reviews processed: 900
The number of negative reviews processed: 900


In [42]:
# Get maximum doc length for padding sequences
max_length = max([len(s.split()) for s in training_docs])
print('The maximum document length:', max_length)

The maximum document length: 1317


In [43]:
# encode training and validation datasets
X_train, vocab_tokenizer = encode_train_data(training_docs, max_length)
print('The shape of the encoded training dataset:', X_train.shape)
y_train = np.array([0 for _ in range(len(negative_train_cases))] + [1 for _ in range(len(positive_train_cases))])
print('The shape of the encoded training classes:', y_train.shape)

The shape of the encoded training dataset: (1800, 1317)
The shape of the encoded training classes: (1800,)


In [44]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 3 - Define and Train Models

In [45]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [46]:
# Define the default numbers of input/output for modeling
num_outputs = 1

In [47]:
# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
    return embedding

In [48]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, default_vector_space))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

In [49]:
# load embedding from file
raw_embedding = load_embedding('embedding_word2vec.txt')

# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, vocab_tokenizer.word_index)

# define vocabulary size (largest integer value)
vocab_size = len(vocab_tokenizer.word_index) + 1
print('The maximum vocabulary size is:', vocab_size)

# create the embedding layer
embedding_layer = keras.layers.Embedding(vocab_size, default_vector_space, weights=[embedding_vectors], input_length=max_length, trainable=False)

The maximum vocabulary size is: 25768


In [50]:
# Define the baseline model for benchmarking
def create_nn_model(n_inputs=max_length, n_outputs=num_outputs, embed_layer=embedding_layer, conv1_filters=default_filters, conv1_kernels=default_kernel_size, 
                    n_pools=default_pool_size, opt_param=default_optimizer, init_param=default_kernel_init):
    nn_model = keras.Sequential([
        embed_layer,
        layers.Conv1D(filters=conv1_filters, kernel_size=conv1_kernels, activation='relu'),
        layers.MaxPooling1D(pool_size=n_pools),
        layers.Flatten(),
        layers.Dense(n_outputs, activation='sigmoid', kernel_initializer=init_param)
    ])
    nn_model.compile(loss=default_loss, optimizer=opt_param, metrics=default_metrics)
    return nn_model

In [51]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=n_folds, n_repeats=n_iterations, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    baseline_model = create_nn_model()
    baseline_model.fit(feature_train, target_train, epochs=default_epochs, batch_size=default_batch_size, verbose=1)
    model_metric = baseline_model.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100),'\n')
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all validation iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Epoch 1/10
90/90 [==============================] - 9s 105ms/step - loss: 0.8665 - accuracy: 0.5090
Epoch 2/10
90/90 [==============================] - 10s 106ms/step - loss: 0.6149 - accuracy: 0.6604
Epoch 3/10
90/90 [==============================] - 9s 105ms/step - loss: 0.5299 - accuracy: 0.7312
Epoch 4/10
90/90 [==============================] - 9s 104ms/step - loss: 0.4212 - accuracy: 0.8014
Epoch 5/10
90/90 [==============================] - 9s 104ms/step - loss: 0.3036 - accuracy: 0.8958
Epoch 6/10
90/90 [==============================] - 10s 106ms/step - loss: 0.2057 - accuracy: 0.9389
Epoch 7/10
90/90 [==============================] - 10s 107ms/step - loss: 0.1297 - accuracy: 0.9750
Epoch 8/10
90/90 [==============================] - 10s 106ms/step - loss: 0.0871 - accuracy: 0.9937
Epoch 9/10
90/90 [==============================] - 10s 107ms/step - loss: 0.0533 - accuracy: 0.9979
Epoch 10/10
90/90 [==============================] - 10s 110ms/step - loss: 0.0331 - accuracy: 

In [52]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 4 - Evaluate and Optimize Models

In [53]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [54]:
# Not applicable for this iteration of modeling

In [55]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 5 - Finalize Model and Make Predictions

In [56]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [57]:
# prepare encoding of docs
def encode_test_data(test_docs, maxlen, tokenizer):
    # encode training data set
    encoded_docs = tokenizer.texts_to_sequences(test_docs)
    # pad sequences
    test_encoded = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
    return test_encoded

In [58]:
# load all validation cases
positive_test_cases = process_docs_to_lines(testing_classA_dir, vocab)
print('The number of positive reviews processed:', len(positive_test_cases))
negative_test_cases = process_docs_to_lines(testing_classB_dir, vocab)
print('The number of negative reviews processed:', len(negative_test_cases))
testing_docs = negative_test_cases + positive_test_cases

The number of positive reviews processed: 100
The number of negative reviews processed: 100


In [59]:
# encode training and validation datasets
X_test = encode_test_data(testing_docs, max_length, vocab_tokenizer)
print('The shape of the encoded test dataset:', X_test.shape)
y_test = np.array([0 for _ in range(len(negative_test_cases))] + [1 for _ in range(len(positive_test_cases))])
print('The shape of the encoded test classes:', y_test.shape)

The shape of the encoded test dataset: (200, 1317)
The shape of the encoded test classes: (200,)


In [60]:
# Create the final model for evaluating the test dataset
reset_random(seedNum)
final_model = create_nn_model()
final_model.fit(X_train, y_train, epochs=default_epochs, batch_size=default_batch_size, verbose=1)

Epoch 1/10
113/113 [==============================] - 12s 106ms/step - loss: 0.8457 - accuracy: 0.5361
Epoch 2/10
113/113 [==============================] - 12s 104ms/step - loss: 0.6687 - accuracy: 0.5928
Epoch 3/10
113/113 [==============================] - 12s 105ms/step - loss: 0.6015 - accuracy: 0.6717
Epoch 4/10
113/113 [==============================] - 12s 106ms/step - loss: 0.5112 - accuracy: 0.7467
Epoch 5/10
113/113 [==============================] - 12s 108ms/step - loss: 0.4084 - accuracy: 0.8183
Epoch 6/10
113/113 [==============================] - 12s 107ms/step - loss: 0.3221 - accuracy: 0.8706
Epoch 7/10
113/113 [==============================] - 12s 107ms/step - loss: 0.2427 - accuracy: 0.9028
Epoch 8/10
113/113 [==============================] - 12s 106ms/step - loss: 0.1858 - accuracy: 0.9394
Epoch 9/10
113/113 [==============================] - 12s 105ms/step - loss: 0.1389 - accuracy: 0.9667
Epoch 10/10
113/113 [==============================] - 12s 107ms/step - l

In [61]:
# Summarize the final model
final_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1313, 128)         64128     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 656, 128)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 83968)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 83969     
Total params: 2,724,897
Trainable params: 148,097
Non-trainable params: 2,576,800
_________________________________________________________________


In [62]:
# test_predictions = final_model.predict(X_test, batch_size=default_batch, verbose=1)
test_predictions = (final_model.predict(X_test) > 0.5).astype("int32").ravel()
print('Accuracy Score:', accuracy_score(y_test, test_predictions))
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

Accuracy Score: 0.49
[[50 50]
 [52 48]]
              precision    recall  f1-score   support

           0       0.49      0.50      0.50       100
           1       0.49      0.48      0.48       100

    accuracy                           0.49       200
   macro avg       0.49      0.49      0.49       200
weighted avg       0.49      0.49      0.49       200



In [63]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [64]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:11:48.403900
